In [ ]:
!pip install langchain_nvidia_ai_endpoints

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.2.0 which is incompatible.


In [12]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA

In [ ]:
!pip install langchain-community langchain-text-splitters faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [7]:
from google.colab import userdata
import os
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')

In [8]:
embeddings = NVIDIAEmbeddings()

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


model = ChatNVIDIA(model="mistral_7b")

In [10]:
model

ChatNVIDIA(client=NVEModel(base_url='https://api.nvcf.nvidia.com/v2/nvcf', get_session_fn=<class 'requests.sessions.Session'>, get_asession_fn=<class 'aiohttp.client.ClientSession'>, endpoints={'infer': '{base_url}/pexec/functions/{model_id}', 'status': '{base_url}/pexec/status/{request_id}', 'models': '{base_url}/functions'}, api_key=SecretStr('**********'), is_staging=False, timeout=60, interval=0.02, last_inputs={}, last_response=None, payload_fn=<function NVEModel.<lambda> at 0x79dfe6285a20>, headers_tmpl={'call': {'Accept': 'application/json', 'Authorization': 'Bearer {api_key}', 'User-Agent': 'langchain-nvidia-ai-endpoints'}, 'stream': {'Accept': 'text/event-stream', 'content-type': 'application/json', 'Authorization': 'Bearer {api_key}', 'User-Agent': 'langchain-nvidia-ai-endpoints'}}, stagify=functools.partial(<function NVEModel._stagify at 0x79dfe6286200>, is_staging=False)), model='mistral_7b')

In [13]:
hyde_template = """Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:

{question}"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_query_transformer = hyde_prompt | model | StrOutputParser()

In [16]:
from langchain_core.runnables import chain

@chain
def hyde_retriever(question):
    hypothetical_document = hyde_query_transformer.invoke({"question": question})
    return retriever.invoke(hypothetical_document)

In [17]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
answer_chain = prompt | model | StrOutputParser()

In [18]:
@chain
def final_chain(question):
    documents = hyde_retriever.invoke(question)
    for s in answer_chain.stream({"question": question, "context": documents}):
        yield s

In [19]:
for s in final_chain.stream("how can langsmith help with testing"):
    print(s, end="")

Langsmith is a platform designed to help with LLM (Language Model) application development, monitoring, and testing. It provides several workflows that can be utilized during different stages of the application development lifecycle.

One of the ways Langsmith can help with testing is by allowing developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. This can help in tracking regressions and improvements, as well as in the development of automatic evaluations.

Another way Langsmith can assist with testing is by allowing users to gather human feedback on the responses produced by the application. This can help draw attention to interesting runs and highlight edge cases causing problematic responses. Langsmith also allows attaching feedback scores to these runs.

Lastly, when developing new LLM applications, Langsmith suggests having tracing enabled by default. This gives clear visibility and debugging 